The following functions are used send queries to Translator and display the results

In [12]:
#imports
import json
import requests
from collections import defaultdict
import pandas as pd
import copy
from datetime import datetime as dt
import urllib.parse
import time

In [13]:
#ARS functions
def submit_to_ars(m,ars_url='https://ars.transltr.io/ars/api',arax_url='https://arax.ncats.io'):
    submit_url=f'{ars_url}/submit'
    response = requests.post(submit_url,json=m)
    try:
        message_id = response.json()['pk']
    except:
        print('fail')
        message_id = None
    print(f'{arax_url}/?source=ARS&id={message_id}')
    return message_id

def retrieve_ars_results(mid,ars_url='https://ars.transltr.io/ars/api'):
    message_url = f'{ars_url}/messages/{mid}?trace=y'
    response = requests.get(message_url)
    j = response.json()
    print( j['status'] )
    results = {}
    for child in j['children']:
        print(child['status'])
        if child['status']  == 'Done':
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                nresults = len(child_response['fields']['data']['message']['results'])
                if nresults > 0:
                    results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
            except Exception as e:
                nresults=0
                child['status'] = 'ARS Error'
        elif child['status'] == 'Error':
            nresults=0
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
            except Exception as e:
                print(e)
                child['status'] = 'ARS Error'
        else:
            nresults = 0
        print( child['status'], child['actor']['agent'],nresults )
    return results

In [14]:
#utils
def printjson(j):
    print(json.dumps(j,indent=4))
def print_json(j):
    printjson(j)

In [15]:
#Method to construct a simple one hop query.  Default values are set to the most general form
def construct_query(id0, type0=["biolink:NamedThing"],type1=["biolink:NamedThing"],
                    predicates=["biolink:related_to"]):
    with open('template.json','r') as inf:
        query = json.load(inf)
        query["message"]["query_graph"]["edges"]["e01"]["predicates"]=predicates
        query["message"]["query_graph"]["nodes"]["n0"]["ids"]=id0
        #query["message"]["query_graph"]["nodes"]["n1"]["ids"]=id1
        query["message"]["query_graph"]["nodes"]["n0"]["categories"]=type0
        query["message"]["query_graph"]["nodes"]["n1"]["categories"]=type1
        return query

In [16]:
def resolve_name(string):
    url_string=urllib.parse.quote(string)
    name_resolver_url="https://name-resolution-sri.renci.org/lookup?string="
    message_url = f'{name_resolver_url}{string}&offset=0&limit=10'
    response = requests.post(message_url)
    return response.json()

In [17]:
def construct_batch(ids0, id1=[], type0=["biolink:NamedThing"],type1=["biolink:NamedThing"],
                    predicates=["biolink:related_to"]):
    query_list=[]
    for my_id in ids0:
        query_list.append(construct_query(my_id,id1,type0,type1,predicates))
    return query_list

In [18]:
def run_batch(query_list,delay=0)
    id_list=[]
    for query in query_list:
        id_list.append(submit_to_ars(query))
        time.sleep(delay)
    return id_list

SyntaxError: invalid syntax (1132795205.py, line 1)

In [19]:
def return_batch(id_list):
    result_list=[]
    for my_id in id_list:
        result=retrieve_ars_results(my_id)
        result_list.append(result)
        time.sleep(delay)
    return result_list
        

In [20]:
def getpath_impl(j, fields, i):
    if(j is None or i>=len(fields)):
        return j
    field = fields[i]
    jNext = j[field] if field in j else None
    return getpath_impl(jNext, fields, i+1)

def getpath(j, fields):
    return getpath_impl(j, fields, 0)

In [80]:
def fetch_triple_from_kg(message,triple):
    kg = getpath(message,["message","knowledge_graph"])
    nodes = getpath(kg,["nodes"])
    edges = getpath(kg,["edges"])
    #print("Nodes "+str(nodes))
    node1=(getpath(nodes,[triple[0],"name"]))
    edge=(getpath(edges,[triple[1],"predicate"]))
    node2=(getpath(nodes,[triple[2],"name"]))
    if node1 is None:
        node1="Not Found"
    if edge is None:
        edge="Not Found"
    if node2 is None:
        node2="Not Found"
    fetched_triple=(node1,edge,node2)

    #print("Fetched "+str(fetched_triple))
    return fetched_triple
    #csv=getpath(nodes,[triple[0],"name"])+","+getpath(edges,[triple[1],"name"])+","+getpath(nodes,[triple[2],"name"])
    #print(csv)
        

def trapi_to_csv(message):
    results = getpath(message,["message","results"])
    #printjson(results)
    kg = getpath(message,["fields","data","message","knowledge_graph"])
    triples=[]
    csv_list=[]
    if results is None:
        #print("No results?")
        return None
    for result in results:
        #print("Result"+ str(result))
        node_bindings=getpath(result,["node_bindings"])
        edge_bindings=getpath(result,["edge_bindings"])
        node_ids=[]
        edge_ids=[]
        for binding in node_bindings:
            
            node_id=node_bindings[binding][0]["id"]
            node_ids.append(node_id)
        for binding in edge_bindings:
            edge_id=edge_bindings[binding][0]["id"]
            edge_ids.append(edge_id)
        #here we enter the realm of assumptions, namely that we are only working with one-hop queries
        my_tuple=(node_ids[0],edge_ids[0],node_ids[1])
        #print("TUPLE "+str(my_tuple))
        f_t=fetch_triple_from_kg(message,my_tuple)
        csv=f_t[0]+","+f_t[1]+","+f_t[2]
        csv_list.append(csv)
    return csv_list

        

In [22]:
 with open('exampleAnswer.json','r') as inf:
        trapi_to_csv(json.load(inf))

In [23]:
#my_s = "pancreatic cancer"
#resolved=resolve_name(my_s)
#printjson(resolved)

In [24]:
my_input ="NCBIGene:1956"
my_query = construct_query([my_input],[],["biolink:Gene"],["biolink:ChemicalEntity"])
print(my_query)

{'message': {'query_graph': {'edges': {'e01': {'object': 'n0', 'subject': 'n1', 'predicates': ['biolink:ChemicalEntity']}}, 'nodes': {'n0': {'ids': ['NCBIGene:1956'], 'categories': []}, 'n1': {'categories': ['biolink:Gene']}}}}}


In [53]:
my_query = construct_query(["NCBIGene:23221"],["biolink:Gene"],["biolink:ChemicalEntity"])
printjson(my_query)
#my_id=submit_to_ars(my_query)
#time.sleep(300)
my_id="1a1aa294-a854-4857-a58c-ef5dfb273fa9"
my_json=retrieve_ars_results(my_id)

{
    "message": {
        "query_graph": {
            "edges": {
                "e01": {
                    "object": "n0",
                    "subject": "n1",
                    "predicates": [
                        "biolink:related_to"
                    ]
                }
            },
            "nodes": {
                "n0": {
                    "ids": [
                        "NCBIGene:23221"
                    ],
                    "categories": [
                        "biolink:Gene"
                    ]
                },
                "n1": {
                    "categories": [
                        "biolink:ChemicalEntity"
                    ]
                }
            }
        }
    }
}
Done
Done
Done ara-aragorn 5000
Done
Done ara-arax 500
Done
Done ara-bte 3451
Error
Error ara-unsecret 0
Done
Done kp-genetics 8
Done
Done kp-molecular 18
Done
Done ara-explanatory 317
Error
Error ara-improving 0
Done
Done kp-cam 0
Done
Done kp-textmining 40
Don

In [83]:
LIMIT =5000
#print(type(my_json))
#print(my_json.keys())
#printjson(my_json["kp-genetics"])
#csvs=trapi_to_csv(my_json["kp-genetics"])
#for csv in csvs:
#    print(str(csv))

for key in my_json.keys():
    print("Currently viewing results from "+key+"\n")
    #print(my_json[result])
    csvs=trapi_to_csv(my_json[key])
    if csvs is None:
        print("None CSVs")
        continue
    counter=0
    for csv in csvs:
        counter+=1
        print(csv)
        if(counter>LIMIT):
            break

Currently viewing results from ara-aragorn

RHOBTB2,biolink:interacts_with,ARHG4_HUMAN
RHOBTB2,biolink:interacts_with,ARHG4_HUMAN
RHOBTB2,biolink:interacts_with,ARHG4_HUMAN
RHOBTB2,biolink:interacts_with,ARHG4_HUMAN
RHOBTB2,biolink:interacts_with,ARHG4_HUMAN
RHOBTB2,biolink:interacts_with,ARHG4_HUMAN
RHOBTB2,biolink:interacts_with,ARHG4_HUMAN
RHOBTB2,biolink:interacts_with,ARHG4_HUMAN
RHOBTB2,biolink:interacts_with,ARHG4_HUMAN
RHOBTB2,biolink:interacts_with,ARHG4_HUMAN
RHOBTB2,biolink:interacts_with,ARHG4_HUMAN
RHOBTB2,biolink:interacts_with,ARHG4_HUMAN
RHOBTB2,biolink:interacts_with,ARHG4_HUMAN
RHOBTB2,biolink:interacts_with,ARHG4_HUMAN
RHOBTB2,biolink:interacts_with,ARHG4_HUMAN
RHOBTB2,biolink:interacts_with,ARHG4_HUMAN
RHOBTB2,biolink:interacts_with,ARHG4_HUMAN
RHOBTB2,biolink:interacts_with,ARHG4_HUMAN
RHOBTB2,biolink:interacts_with,ARHG4_HUMAN
RHOBTB2,biolink:interacts_with,ARHG4_HUMAN
RHOBTB2,biolink:interacts_with,ARHG4_HUMAN
RHOBTB2,biolink:interacts_with,ARHG4_HUMAN
RHOBTB2,bi

RHOBTB2,biolink:is_nearby_variant_of,rs76824703
RHOBTB2,biolink:is_nearby_variant_of,rs13263698
RHOBTB2,biolink:is_nearby_variant_of,rs35788063
RHOBTB2,biolink:is_nearby_variant_of,rs4281113
RHOBTB2,biolink:is_nearby_variant_of,rs4871856
RHOBTB2,biolink:is_nearby_variant_of,rs1010156
RHOBTB2,biolink:is_nearby_variant_of,rs79948702
RHOBTB2,biolink:is_nearby_variant_of,rs12544038
RHOBTB2,biolink:is_nearby_variant_of,rs4872050
RHOBTB2,biolink:is_nearby_variant_of,rs17089295
RHOBTB2,biolink:is_nearby_variant_of,rs72607402
RHOBTB2,biolink:is_nearby_variant_of,rs73229817
RHOBTB2,biolink:is_nearby_variant_of,rs7004710
RHOBTB2,biolink:entity_negatively_regulates_entity,myometrium
RHOBTB2,biolink:is_nearby_variant_of,rs2457431
RHOBTB2,biolink:is_nearby_variant_of,rs6999825
RHOBTB2,biolink:is_nearby_variant_of,rs883886
RHOBTB2,biolink:is_nearby_variant_of,rs10092352
RHOBTB2,biolink:is_nearby_variant_of,rs7017737
RHOBTB2,biolink:is_nearby_variant_of,rs12546582
RHOBTB2,biolink:is_nearby_variant_of

RHOBTB2,biolink:is_nearby_variant_of,rs73551901
RHOBTB2,biolink:is_nearby_variant_of,rs56160478
RHOBTB2,biolink:is_nearby_variant_of,rs369184207
RHOBTB2,biolink:is_nearby_variant_of,rs76856898
RHOBTB2,biolink:expression_increased_by,rs1051647
RHOBTB2,biolink:is_nearby_variant_of,rs3758040
RHOBTB2,biolink:is_nearby_variant_of,rs6557604
RHOBTB2,biolink:is_nearby_variant_of,rs7841172
RHOBTB2,biolink:is_nearby_variant_of,rs73215076
RHOBTB2,biolink:is_nearby_variant_of,rs10503718
RHOBTB2,biolink:is_nearby_variant_of,rs6981627
RHOBTB2,biolink:is_nearby_variant_of,rs13268466
RHOBTB2,biolink:is_nearby_variant_of,rs4422779
RHOBTB2,biolink:is_nearby_variant_of,rs2404656
RHOBTB2,biolink:is_nearby_variant_of,rs4872035
RHOBTB2,biolink:is_nearby_variant_of,rs35742726
RHOBTB2,biolink:is_nearby_variant_of,rs140991613
RHOBTB2,biolink:is_nearby_variant_of,rs7460036
RHOBTB2,biolink:is_nearby_variant_of,rs7001838
RHOBTB2,biolink:is_nearby_variant_of,rs376689403
RHOBTB2,biolink:is_nearby_variant_of,rs69951

RHOBTB2,biolink:expresses,descending thoracic aorta
RHOBTB2,biolink:expresses,thoracic aorta
RHOBTB2,biolink:participates_in,supramolecular fiber organization
RHOBTB2,biolink:interacts_with,RHG06_HUMAN
RHOBTB2,biolink:orthologous_to,racC
RHOBTB2,biolink:entity_positively_regulates_entity,left lobe of thyroid gland
RHOBTB2,biolink:has_real_world_evidence_of_association_with,ICE1
RHOBTB2,biolink:interacts_with,RGRF2_HUMAN
RHOBTB2,biolink:interacts_with,ERLOTINIB
DBSNP:rs1377944151,biolink:is_sequence_variant_of,RHOBTB2
RHOBTB2,biolink:interacts_with,RHG12_HUMAN
RHOBTB2,biolink:increases_expression_of,calcium;silicate
RHOBTB2,biolink:is_nearby_variant_of,rs3857977
RHOBTB2,biolink:expresses,caudate nucleus
RHOBTB2,biolink:interacts_with,GNA13_HUMAN
RHOBTB2,biolink:entity_negatively_regulates_entity,TNFRSF21
RHOBTB2,biolink:participates_in,phagocytosis, engulfment
Lyme disease,biolink:condition_associated_with_gene,RHOBTB2
RHOBTB2,biolink:entity_negatively_regulates_entity,blood
N-[3-[[5-Br

CAID:CA12833030,biolink:is_nearby_variant_of,RHOBTB2
CAID:CA12718489,biolink:is_nearby_variant_of,RHOBTB2
CAID:CA174006543,biolink:is_nearby_variant_of,RHOBTB2
CAID:CA173873376,biolink:is_nearby_variant_of,RHOBTB2
CAID:CA173856668,biolink:is_nearby_variant_of,RHOBTB2
CAID:CA12914110,biolink:is_nearby_variant_of,RHOBTB2
CAID:CA174006369,biolink:is_nearby_variant_of,RHOBTB2
CAID:CA173991994,biolink:is_nearby_variant_of,RHOBTB2
CAID:CA12877016,biolink:is_nearby_variant_of,RHOBTB2
CAID:CA12833305,biolink:is_nearby_variant_of,RHOBTB2
CAID:CA12833176,biolink:is_nearby_variant_of,RHOBTB2
CAID:CA174007381,biolink:is_nearby_variant_of,RHOBTB2
CAID:CA12877040,biolink:is_nearby_variant_of,RHOBTB2
CAID:CA12756500,biolink:is_nearby_variant_of,RHOBTB2
CAID:CA16329429,biolink:is_nearby_variant_of,RHOBTB2
CAID:CA16329430,biolink:is_nearby_variant_of,RHOBTB2
CAID:CA12718443,biolink:is_nearby_variant_of,RHOBTB2
CAID:CA12718444,biolink:is_nearby_variant_of,RHOBTB2
CAID:CA12833023,biolink:is_nearby_varian

CAID:CA4671984,biolink:is_nearby_variant_of,RHOBTB2
CAID:CA12718386,biolink:is_nearby_variant_of,RHOBTB2
CAID:CA173894105,biolink:is_nearby_variant_of,RHOBTB2
CAID:CA12786590,biolink:is_nearby_variant_of,RHOBTB2
CAID:CA12718340,biolink:is_nearby_variant_of,RHOBTB2
CAID:CA12933924,biolink:is_nearby_variant_of,RHOBTB2
CAID:CA4672015,biolink:is_nearby_variant_of,RHOBTB2
CAID:CA12786592,biolink:is_nearby_variant_of,RHOBTB2
CAID:CA12786591,biolink:is_nearby_variant_of,RHOBTB2
CAID:CA12903672,biolink:is_nearby_variant_of,RHOBTB2
CAID:CA12718300,biolink:is_nearby_variant_of,RHOBTB2
CAID:CA4667948,biolink:is_nearby_variant_of,RHOBTB2
CAID:CA12786594,biolink:is_nearby_variant_of,RHOBTB2
CAID:CA173898079,biolink:is_nearby_variant_of,RHOBTB2
CAID:CA12786595,biolink:is_nearby_variant_of,RHOBTB2
CAID:CA12808144,biolink:is_nearby_variant_of,RHOBTB2
CAID:CA12876996,biolink:is_nearby_variant_of,RHOBTB2
CAID:CA12718343,biolink:is_nearby_variant_of,RHOBTB2
CAID:CA173966600,biolink:is_nearby_variant_of,R

RHOBTB2,biolink:interacts_with,VINORELBINE
RHOBTB2,biolink:interacts_with,IRINOTECAN
RHOBTB2,biolink:interacts_with,ERLOTINIB
RHOBTB2,biolink:interacts_with,DOCETAXEL
RHOBTB2,biolink:interacts_with,BEVACIZUMAB
RHOBTB2,biolink:interacts_with,ALIMTA
RHOBTB2,biolink:interacts_with,HSPPC-96 VACCINE
RHOBTB2,biolink:genetically_interacts_with,SSPO
RHOBTB2,biolink:genetically_interacts_with,LAMA1
RHOBTB2,biolink:genetically_interacts_with,CSMD2
RHOBTB2,biolink:genetically_interacts_with,INSRR
RHOBTB2,biolink:genetically_interacts_with,VCAN
RHOBTB2,biolink:genetically_interacts_with,C18orf8
RHOBTB2,biolink:genetically_interacts_with,BRCA1
RHOBTB2,biolink:genetically_interacts_with,WNK3
RHOBTB2,biolink:genetically_interacts_with,NBEA
RHOBTB2,biolink:genetically_interacts_with,ICE1
RHOBTB2,biolink:has_real_world_evidence_of_association_with,SSPO
RHOBTB2,biolink:has_real_world_evidence_of_association_with,LAMA1
RHOBTB2,biolink:has_real_world_evidence_of_association_with,CSMD2
RHOBTB2,biolink:has_

In [6]:
with open('A.0_RHOBTB2_direct.json','r') as inf:
    query_0 = json.load(inf)
printjson(query_0)

{
    "message": {
        "query_graph": {
            "edges": {
                "e01": {
                    "object": "n0",
                    "subject": "n1",
                    "predicates": [
                        "biolink:entity_negatively_regulates_entity",
                        "biolink:decreases_abundance_of",
                        "biolink:decreases_expression_of",
                        "biolink:decreases_stability_of",
                        "biolink:decreases_uptake_of",
                        "biolink:increases_degradation_of",
                        "biolink:decreases_synthesis_of",
                        "biolink:decreases_activity_of"
                    ]
                }
            },
            "nodes": {
                "n0": {
                    "ids": [
                        "NCBIGene:23221"
                    ],
                    "categories": [
                        "biolink:Gene"
                    ]
                },
              

In [ ]:
pk_0 = submit_to_ars(query_0)